In [2]:
from folium.plugins import HeatMapWithTime
import pandas as pd
import numpy as np
import folium

In [3]:
lat_lon_df = pd.read_csv("../data/pref_lat_lon.csv")
population_2020_df = pd.read_csv("../data/population_2020.csv")
confirmed_cases_daily_df = pd.read_csv("../data/newly_confirmed_cases_daily.csv")

tmp_df = pd.merge(confirmed_cases_daily_df, lat_lon_df, on="pref_name")
merged_df = pd.merge(tmp_df, population_2020_df, on="pref_name")

In [4]:
# 時系列のカラム名の配列を取得する
time_columns = merged_df.columns[2:1210].values

time_columns

array(['2020-1-17', '2020-1-18', '2020-1-19', ..., '2023-5-6', '2023-5-7',
       '2023-5-8'], dtype=object)

In [10]:
# diff_dataをスケーリングし、scaled_dataとする
scaled_data = merged_df.copy()


for i in range(0, 46):
    scaled_data.loc[:, time_columns] = (
        merged_df.loc[:, time_columns] / merged_df["population"].loc[i] + 1e-4
    ) * 1000

/var/folders/k8/xq9_8lkd7bqc5dx7rlkdm0qr0000gn/T/ipykernel_55562/2289047416.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  scaled_data.loc[:, time_columns] = (
/var/folders/k8/xq9_8lkd7bqc5dx7rlkdm0qr0000gn/T/ipykernel_55562/2289047416.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  scaled_

In [11]:
heat_map_data = [
    [[row["lat"], row["lon"], row[idx]] for index, row in scaled_data.iterrows()]
    for idx in time_columns
]

In [7]:
japan_map = folium.Map(location=[35, 135], zoom_start=6)
hm = HeatMapWithTime(
    heat_map_data,
    index=list(time_columns),
    auto_play=False,
    radius=30,
    max_opacity=1,
    gradient={0.1: "blue", 0.25: "lime", 0.5: "yellow", 0.75: "orange", 0.9: "red"},
)
hm.add_to(japan_map)

japan_map.save("../data/heatmap.html")

japan_map